In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns
import scipy.stats as sts

import random
import string

In [ ]:
from nb_vars import FOLDER_OUTPUT_BASE, CUTOFF_SPIKES_HZ

## Load df of filtered peaks

In [ ]:
CONDITION_NAME = "Placa"
CONDITION = "1+2"

folder_input =  f'{FOLDER_OUTPUT_BASE}/{CONDITION_NAME}_{CONDITION}'

In [ ]:
df_peaks = pd.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv')

In [ ]:
df_peaks

## Relative change in MFR

In [ ]:
df_MFR = df_peaks.groupby(['condition', 'treatment', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks['time'].max() - df_peaks['time'].min())
df_MFR = df_MFR.reset_index().sort_values(by=['well', 'electrode'])
df_MFR = df_MFR.rename(columns={'time': 'MFR'})
df_MFR

In [ ]:
df_MFR[df_MFR['treatment'] == 'POST']

In [ ]:
df_MFR_ratio = df_MFR[df_MFR['treatment'] == 'POST']

ratios = df_MFR[df_MFR['treatment'] == 'POST']['MFR'].values /  df_MFR[df_MFR['treatment'] == 'PRE']['MFR'].values

df_MFR_ratio.loc[:, 'PRE'] = df_MFR[df_MFR['treatment'] == 'PRE'].loc[:, 'MFR'].values
df_MFR_ratio.loc[:, 'POST'] = df_MFR[df_MFR['treatment'] == 'POST'].loc[:, 'MFR'].values
df_MFR_ratio['mean_PRE_POST'] = (df_MFR_ratio['PRE'] + df_MFR_ratio['POST']) / 2

df_MFR_ratio = df_MFR_ratio.rename(columns={'MFR': 'MFR_ratio'})[['condition', 'well', 'replicate', 'electrode', 'PRE', 'POST', 'mean_PRE_POST', 'MFR_ratio']]
df_MFR_ratio['MFR_ratio'] = ratios * 100

df_MFR_ratio

df_MFR_ratio.to_csv(f'{folder_input}/MFR_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv', sep=';', index=None)

In [ ]:
fig = plt.figure(figsize=(29,6))
sns.swarmplot(data = df_MFR_ratio, x='condition', y='MFR_ratio', hue='replicate')
plt.plot([-0.5, len(df_MFR_ratio['condition'].drop_duplicates()) - 0.5], [100, 100], c='#bc0000')

In [ ]:
fig = plt.figure(figsize=(29,6))
sns.boxplot(data = df_MFR_ratio, x='condition', y='MFR_ratio')
plt.plot([-0.5, len(df_MFR_ratio['condition'].drop_duplicates()) - 0.5], [100, 100], c='#bc0000')

In [ ]:
sns.scatterplot(data = df_MFR_ratio, x='PRE', y='MFR_ratio', label='PRE')
sns.scatterplot(data = df_MFR_ratio, x='POST', y='MFR_ratio', label='POST')

In [ ]:
#create regplot
p = sns.regplot(data = df_MFR_ratio, x='mean_PRE_POST', y='MFR_ratio')

#calculate slope and intercept of regression equation
slope, intercept, r, p, sterr = sts.linregress(x=p.get_lines()[0].get_xdata(),
                                                       y=p.get_lines()[0].get_ydata())

#display slope and intercept of regression equation
print(intercept, slope, p)
